In [16]:
import numpy as np
import pandas
import requests
from bs4 import BeautifulSoup as bs
from IPython.display import HTML
import warnings
from datetime import date
import random
import matplotlib.pyplot as plt

In [17]:
#Scraper
for i in (0,20,40,60):
  if(i==0):
      url="https://zk.mk/umetnichki-galerii"
  else:
    url="https://zk.mk/umetnichki-galerii?skip="+str(i)
  response = requests.get(url)
  html = response.text
  bshtml= bs(html,"html.parser")
  galeries = bshtml.select('a.companyname')
  galeries = [item.text for item in galeries]
  grad = bshtml.select('li.h4efd8293a5270db14490d2fba73e0d16')
  grad = [item.text.split(":")[1].strip() for item in grad]
  desc = bshtml.select('div.shortdescription')
  desc= desc[1::2]
  desc = [item.text.split(":")[1] for item in desc]
  adresa = bshtml.select('li.h56146bf7196e9cd84da16cf2ccd2a8df')
  adresa = [item.text.split(":")[1].strip() for item in adresa]
  rabvreme = bshtml.select('li.ha406382314ab32830b5be3ba354edd06')
  rabvreme = [item.text.split(":")[1].strip() for item in rabvreme]
  rabvreme.extend([''] * (len(galeries)-len(rabvreme)))
  pauza = bshtml.select('li.h0d9b72a41fb93f4d82f8485abd5aba85')
  pauza = [item.text.split(":")[1].strip() for item in pauza]
  pauza.extend([''] * (len(galeries)-len(pauza)))

  items={}
  items["Umetnicka galerija"]=galeries
  items["Adresa"]=adresa
  items["Grad"]=grad
  items["Rabotno Vreme"]=rabvreme
  items["Pauza"]=pauza
  items["Opis"]=desc

  umetnicki_galerii=pandas.DataFrame(items)
  if(i==0):
    umetnicki_galerii.to_csv('Galerii.csv', index=False)
  else:
    umetnicki_galerii.to_csv('Galerii.csv',mode='a', index=False, header=False)

galerii_prefilter= pandas.read_csv('/content/Galerii.csv')


In [18]:
#get the file
galerii_prefilter= pandas.read_csv('/content/Galerii.csv')

In [19]:
#Pipe
class Pipe:
    def __init__(self):
        self.filters = list()

    def add(self, filter):
        self.filters.append(filter)

    def execute(self, message):
        print("Executing pipeline...")
        for message_filter in self.filters:
            print('Filtering with',message_filter)
            message=message_filter(message)
        print("Done.")
        return message

In [22]:
#Filters
def drop_columns(data):
  return data.drop(columns=['Rabotno Vreme', 'Pauza'])


In [24]:
pipe=Pipe()
pipe.add(drop_columns)
Galerii_filtered=pipe.execute(galerii_prefilter)
Galerii_filtered.to_csv("Galerii_Filtered")

Executing pipeline...
Filtering with <function drop_columns at 0x787547e1dd80>
Done.
